# 3. Handling Disruptions in Workforce Allocation

In the previous notebook, we found an optimal solution to the Workforce Allocation Problem. However, real-world operations are rarely static. Disruptions—such as a team becoming unavailable can make your current plan/allocation obsolete and even infeasible.

This notebook focuses on modeling such disruptions. We will use a scenario generator to create modified problem instances from our original plan. We will see how some disruptions can be accommodated, while others can make the problem **unsatisfiable (UNSAT)**, meaning no solution exists that respects all the rules.

### Prerequisites

In [ ]:
# On Colab: install the library
import sys

ON_COLAB = "google.colab" in sys.modules
if ON_COLAB:
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install discrete-optimization

### Imports

In [ ]:
import copy

from discrete_optimization.datasets import fetch_data_from_cp25
from discrete_optimization.generic_tools.do_solver import StatusSolver
from discrete_optimization.workforce.allocation.parser import (
    build_allocation_problem_from_scheduling,
)

# Import problem and parser
from discrete_optimization.workforce.allocation.problem import (
    AllocationAdditionalConstraint,
)

# Import the CP-SAT solver and the scenario generator
from discrete_optimization.workforce.allocation.solvers.cpsat import (
    CpsatTeamAllocationSolver,
    ModelisationAllocationOrtools,
)

# Import plotting utilities and CP parameters
from discrete_optimization.workforce.allocation.utils import plot_allocation_solution
from discrete_optimization.workforce.generators.resource_scenario import (
    ParamsRandomness,
    generate_allocation_disruption,
)
from discrete_optimization.workforce.scheduling.parser import (
    get_data_available,
    parse_json_to_problem,
)

### Load Original Problem and Optimal Solution

First, we will solve the original problem to get an optimal baseline plan, which will then be disrupted.

In [ ]:
# Load the original problem
files = get_data_available()
if not files:
    fetch_data_from_cp25()
    files = get_data_available()
file_path = files[1]
scheduling_problem = parse_json_to_problem(file_path)
original_problem = build_allocation_problem_from_scheduling(problem=scheduling_problem)

# Solve it to get the optimal baseline solution
solver = CpsatTeamAllocationSolver(original_problem)
solver.init_model(modelisation_allocation=ModelisationAllocationOrtools.BINARY)
result_storage = solver.solve(time_limit=30)
baseline_solution, _ = result_storage.get_best_solution_fit()
baseline_fits = original_problem.evaluate(baseline_solution)
print(f"Baseline optimal solution found with {int(baseline_fits['nb_teams'])} teams.")

## Generating Disruption Scenarios

We will use the `generate_allocation_disruption` function to introduce changes. This function modifies team calendars to simulate unavailability.

### Example 1: A Solvable Disruption

Let's create a minor disruption where one team is unavailable for a short period. The optimizer should be able to find a new solution, possibly by reassigning a few tasks.

In [ ]:
# Parameters for a small disruption
params_small_disruption = ParamsRandomness(
    lower_nb_disruption=1,  # 1 period of unavailability
    upper_nb_disruption=1,
    lower_nb_teams=1,  # for 1 team
    upper_nb_teams=1,
    duration_discrete_distribution=([30], [1.0]),  # 30 minutes long
)

# Generate the disrupted problem
disruption_scenario = generate_allocation_disruption(
    original_allocation_problem=original_problem,
    original_solution=baseline_solution,
    params_randomness=params_small_disruption,
)
disrupted_problem_1 = disruption_scenario["new_allocation_problem"]

# Solve the new problem
solver_1 = CpsatTeamAllocationSolver(disrupted_problem_1)
solver_1.init_model()
result_1 = solver_1.solve(time_limit=10)
solution_1, _ = result_1.get_best_solution_fit()
fitness_1 = original_problem.evaluate(baseline_solution)

print(f"After small disruption:")
print(f"  - Solver Status: {solver_1.status_solver}")
print(f"  - New number of teams: {int(fitness_1['nb_teams'])}")

In [ ]:
# Visualize the new solution along with the minor disruption
if solver_1.status_solver in {StatusSolver.OPTIMAL, StatusSolver.SATISFIED}:
    fig = plot_allocation_solution(
        problem=disrupted_problem_1,
        sol=solution_1,
        title="Corrected allocation problem with ",
        use_color_map=True,
        plot_breaks=True,
        color_break="black",
        display=False,
    )
    fig.show()

### Example 2: An Unsatisfiable (UNSAT) Scenario

Now, let's create a more severe disruption that makes the problem impossible to solve. A simple way to do this is to impose a new business constraint that conflicts with the optimal solution. For example, an equipment might break and lead to use less team than expected. 

In the following cells, we emulate this situation by specifiying to our model that it should find solution with an additional constraint ($nb\_teams\leq max\_teams\_allowed$)



In [ ]:
# The optimal solution required N teams. Let's create a new problem that only allows N-1.

optimal_teams_count = int(baseline_fits["nb_teams"])
max_teams_allowed = optimal_teams_count - 1

# Create a copy of the original problem to add the new constraint
problem_unsat = copy.deepcopy(original_problem)

# Add the new constraint: nb_max_teams
if problem_unsat.allocation_additional_constraint is None:
    problem_unsat.allocation_additional_constraint = AllocationAdditionalConstraint()
problem_unsat.allocation_additional_constraint.nb_max_teams = max_teams_allowed

print(f"Original optimal teams: {optimal_teams_count}")
print(f"New constraint: Maximum teams allowed = {max_teams_allowed}")

# Try to solve this impossible problem
solver_unsat = CpsatTeamAllocationSolver(problem_unsat)
solver_unsat.init_model()
result_unsat = solver_unsat.solve(time_limit=10)

print(f"\nAfter severe disruption:")
print(f"  - Solver Status: {solver_unsat.status_solver}")

As expected, the solver returns an `UNSATISFIABLE` status. It has mathematically proven that no solution exists that can satisfy all tasks, all constraints (overlap, availability), AND the new requirement of using fewer teams.

## Conclusion

We have demonstrated how to model disruptions, either by changing resource availability or by adding new, stricter constraints. We've also seen how this can lead to an infeasible, or UNSAT, problem.

The crucial question for a planner is now: **why** is it infeasible? Which specific combination of tasks and constraints is causing the conflict? Just knowing that a problem is unsolvable isn't enough to make a good business decision.

➡️ **Next up:** In our final [Notebook 4](4_Explaining_Unsatisfiable_Problems.ipynb), we'll use advanced techniques to diagnose the cause of infeasibility and interactively find a compromise solution.